# File creation #

For length and flexibility reasons, the file-creation from the API and the creation of the RAG and RAG-specific file, have been split into two parts. 

The source for all information can be found here: https://www.dnd5eapi.co/ 

In [2]:
# All needed imports
import requests
import pprint
import json
import spacy
from bs4 import BeautifulSoup
import re
import time

In [3]:
# File path variables.
file_path_rag = 'api_data/rag_data.json'
file_path = 'api_data/api_data.json'

# As there is a specific rate limit of 10k requests per second, code to respect that rate limit including a buffer were integrated into every subcategory API-request call:
MAX_REQUESTS_PER_SECOND = 5000
DELAY = 1 / MAX_REQUESTS_PER_SECOND

In [20]:
# Below is the website link and code to generally acces the api and display the different tables that are supposed to get saved.
# This general code was taken from the api-website to get an understanding of the access.
url = "https://www.dnd5eapi.co/api/2014/spells"

payload = {}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
answer_whole = response.text
print(answer_whole)

{"count":319,"results":[{"index":"acid-arrow","name":"Acid Arrow","level":2,"url":"/api/2014/spells/acid-arrow"},{"index":"acid-splash","name":"Acid Splash","level":0,"url":"/api/2014/spells/acid-splash"},{"index":"aid","name":"Aid","level":2,"url":"/api/2014/spells/aid"},{"index":"alarm","name":"Alarm","level":1,"url":"/api/2014/spells/alarm"},{"index":"alter-self","name":"Alter Self","level":2,"url":"/api/2014/spells/alter-self"},{"index":"animal-friendship","name":"Animal Friendship","level":1,"url":"/api/2014/spells/animal-friendship"},{"index":"animal-messenger","name":"Animal Messenger","level":2,"url":"/api/2014/spells/animal-messenger"},{"index":"animal-shapes","name":"Animal Shapes","level":8,"url":"/api/2014/spells/animal-shapes"},{"index":"animate-dead","name":"Animate Dead","level":3,"url":"/api/2014/spells/animate-dead"},{"index":"animate-objects","name":"Animate Objects","level":5,"url":"/api/2014/spells/animate-objects"},{"index":"antilife-shell","name":"Antilife Shell",

In [ ]:
# When looking at some of the textual entries, there were multiple entries containing '#', '\n' and multiple whitespaces so they were all removed.
nlp = spacy.load("en_core_web_sm")
text_p = 'How does this work?'

# This method was taken from our exercise class:
def remove_xml_tags(review_text):
    return BeautifulSoup(review_text, "html.parser").text

# This method was also inspired from the one in our class but changed so if fits the context.
def preprocess_text(text):
    # Some of the handled descriptions were lists of strings, so it was checked whether that was the case for each entry string.
    # If they were in a list, the entries were joined to one single string.
    if isinstance(text,list):
        text = ' '.join(text)  
    # Possible html tags were removed
    free_text = remove_xml_tags(text)

    # The unwanted characters were removed - lowering the text and removing stopwords and punctuation was not done, because the llm later needs to restrucutre the given text into an response,
    # and to keep the 'sense' of the description, the stopwords weren't removed.
    # In order to remove these characters, they were filtered by a regex.
    free_text = re.sub(r"[#_*\\(\)\n]", "", text)
    free_text = re.sub(r"\s{2,}", " ", free_text)
    free_text = re.sub(r"[{2,}-]", " ", free_text)
    return free_text.strip()

In [ ]:
# These tables were all handled at once, because when looking at them, they had the same basic structure:
list_of_indices = ['conditions','damage-types','magic-schools','weapon-properties']
# Empty dictionaries to later store the information were initialized:
dict_of_conditions = {}
dict_of_damage_types = {}
dict_of_magic_schools = {}
dict_of_weapon_properties = {}

# This method takes the str-input that functions as a identifier for the dict and the indexing word.
def create_dict(type):
    # A dictionary that holds the response data.
    dict_of_response_data = {}

    time.sleep(DELAY)
    
    url = "https://www.dnd5eapi.co/api/2014/"+type
    response = requests.request("GET", url, headers=headers, data=payload)
    resp = response.json()
    # Every entry in the results is walked through and information such as 'name' and 'description' is saved in a variable.
    for entry in resp['results']:
        response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
        resp2 = response2.json()
        # They are joined in a dictionary specific for each entry in the response list.
        response_data = {
            'name': entry['name'].lower(),
            'desc': "".join(preprocess_text(resp2['desc']))
        }
        # For every index in the list an dictionary entry is added to the returned dictionary.
        dict_of_response_data[entry['index']] = response_data
    return dict_of_response_data

# All similar dictionaries are created below:
dict_of_conditions = create_dict(list_of_indices[0])
dict_of_damage_types = create_dict(list_of_indices[1])
dict_of_magic_schools = create_dict(list_of_indices[2])
dict_of_weapon_properties = create_dict(list_of_indices[3])
# An example from above:
pprint.pprint(dict_of_conditions)

In [ ]:
# This cell handles the establishment of the trait dictionary. This cell and all the ones below concerning the api follow the same structure in general:
# 1. The information is pulled out.
# 2. The relevant information is saved in a dictionary (entries like 'url' were ignored, because they don't contain relevant information)
# 3. The information for each entry gets saved in a bigger dictionary.

url = "https://www.dnd5eapi.co/api/2014/traits"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_traits = {}
trait_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    trait_data = {
         'name': name,
          'desc': "".join(preprocess_text(resp2['desc']))
    }
     # This structure tries to identify whether certain 'variables' exist in the table,
     # they follow the same structure but some don't have values stored in them and these ones are not supposed to be saved, so there is less data.
    if resp2.get('races'):
         trait_data['races'] = [item['name'] for item in resp2['races']]

    if resp2.get('subraces'):
         trait_data['subraces'] = [item['name'] for item in resp2['subraces']]

    if resp2.get('proficiencies'):
         trait_data['proficiencies'] = [item['name'] for item in resp2['proficiencies']]
     
     # At the end all of indices are saved into the dictionary.
    dict_of_traits[entry['index']] = trait_data
pprint.pprint(dict_of_traits)

In [ ]:
# This cell handles the possible rules and their sections:
url = "https://www.dnd5eapi.co/api/2014/rule-sections"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_rule_sections = {}
section_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)

     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     section_data = {
         'name': name,
         'desc': preprocess_text(resp2['desc']),
    }
     
     dict_of_rule_sections[entry['index']] = section_data
pprint.pprint(dict_of_rule_sections)

In [ ]:
# This cell handles the possible skills:
url = "https://www.dnd5eapi.co/api/2014/skills"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_skills = {}
skill_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)

     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     skill_data = {
         'name': name,
         'desc': preprocess_text(resp2['desc']),
         'ability_score': resp2['ability_score']['name']
    }
     
     dict_of_skills[entry['index']] = skill_data
pprint.pprint(dict_of_skills)

In [ ]:
# This cell handles the possible feats. The API contain in this and the background table only one entry due to Copyright reasons.
url = "https://www.dnd5eapi.co/api/2014/feats"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_feats = {}
feat_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)

     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     feat_data = {
         'name': name,
         'desc': preprocess_text(resp2['desc'])
    }
     if resp2.get('prerequisites'):
          feat_data['prerequisites']= [{'ability_score': item['ability_score']['name'], 'minimum_score':item['minimum_score'] }for item in resp2['prerequisites']]
     
     dict_of_feats[entry['index']] = feat_data
pprint.pprint(dict_of_feats)

In [ ]:
# This cell handles the ability score table from the API:
url = "https://www.dnd5eapi.co/api/2014/ability-scores"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_ability_scores = {}
ability_score_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)
     
     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     ability_score_data = {
         'abbreviation': name,
         'name': resp2['full_name'],
         'desc': preprocess_text(resp2['desc'])
    }
     if resp2.get('skills'):
          ability_score_data['skills']= [item['name'] for item in resp2['skills']]
     
     dict_of_ability_scores[entry['index']] = ability_score_data
pprint.pprint(dict_of_ability_scores)

In [ ]:
# This cel handles the language table from the API:
url = "https://www.dnd5eapi.co/api/2014/languages"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_languages = {}
language_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)
     
     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     language_data = {
         'name': name,
         'type': resp2['type'],
         'typical_speakers': resp2['typical_speakers']
    }
     if resp2.get('script'):
          language_data['script']= resp2['script']
     
     dict_of_languages[entry['index']] = language_data
pprint.pprint(dict_of_languages)

In [ ]:
# This cell handles the different classes there are in the API:
url = "https://www.dnd5eapi.co/api/2014/classes"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_classes = {}
class_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)

     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     class_data = {
         'name': name,
         'hit_die': resp2['hit_die']
    }
     # This structure tries to identify whether certain 'variables' exist in the table,
     # they follow the same structure but some don't have values stored in them and these ones are not supposed to be saved, so there is less data.
     if resp2.get('proficiency_choices'):
         class_data['proficiency_choices'] = preprocess_text([item['desc'] for item in resp2['proficiency_choices']])

     if resp2.get('proficiencies'):
         class_data['proficiencies'] = [item['name'] for item in resp2['proficiencies']]

     if resp2.get('saving_throws'):
         class_data['saving_throws'] = [item['name'] for item in resp2['saving_throws']]
     
     if resp2.get('starting_equipment'):
         class_data['starting_equipment'] = [{'name': item['equipment']['name'], 'quantity': item['quantity']} for item in resp2['starting_equipment']]
     
     if resp2.get('starting_equipment_options'):
         class_data['starting_equipment_options'] = preprocess_text([item['desc'] for item in resp2['starting_equipment_options']])
     

     time.sleep(DELAY)
     
     # This table includes annother link for each class the link to this varaible looks like this f.ex. 'https://www.dnd5eapi.co/api/2014/classes/barbarian/levels'
     response3 = requests.request("GET", url+f"/{entry['index']}/levels", headers=headers, data=payload)
     resp3 = response3.json()

     level_changes = []
     # for every entry there are changes to the character and these changes will be saved in the level_changes list and later added to the
     # class structure above under the key "class_levels":
     for lvl_entries in resp3:
          level_dict = {
               'level': lvl_entries['level'],
               'ability_score_bonuses': lvl_entries['ability_score_bonuses'],
               'proficienciy_bonus': lvl_entries['prof_bonus'],
               'features': [item['name'] for item in lvl_entries['features']],
               'class_specific': lvl_entries['class_specific']
          }
          level_changes.append(level_dict)

     if resp2.get('class_levels'):
         class_data['class_levels'] = level_changes
     
     if resp2.get('multi_classing'):
         multi_class_dict = {}
         if resp2['multi_classing'].get('prerequisites'):
            prerequisites = [{'ability': item['ability_score']['name'], 'minimum_score': item['minimum_score']} for item in resp2['multi_classing']['prerequisites']]
            multi_class_dict['prerequisites'] = prerequisites
         if resp2['multi_classing'].get('proficiencies'):
            multi_class_dict['proficienies'] = [item['name'] for item in resp2['multi_classing']['proficiencies']]
         class_data['multi_classing'] = [multi_class_dict]
     
     if resp2.get('subclasses'):
         class_data['subclasses'] = [item['name'] for item in resp2['subclasses']]
     

     # At the end all of indices are saved into the dictionary.
     dict_of_classes[entry['index']] = class_data
pprint.pprint(dict_of_classes)

In [ ]:
# This cell handles the subclasses the player can be in the API:
url = "https://www.dnd5eapi.co/api/2014/subclasses"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_subclasses = {}
subclass_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(DELAY)

     response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
     resp2 = response2.json()

     subclass_data = {
         'name': name,
         'class': resp2['class']['name'],
         'subclass_flavor': resp2['subclass_flavor'],
         'desc': preprocess_text(resp2['desc'])
    }
     
     time.sleep(DELAY)
     
      # This table includes annother link for each class the link to this varaible looks like this f.ex. 'https://www.dnd5eapi.co/api/2014/classes/barbarian/levels'
     response3 = requests.request("GET", url+f"/{entry['index']}/levels", headers=headers, data=payload)
     resp3 = response3.json()

     sublevel_changes = []

     for lvl_entries in resp3:
          sublevel_dict = {
               'level': lvl_entries['level'],
               'features': [item['name'] for item in lvl_entries['features']]
          }
          sublevel_changes.append(level_dict)

     if resp2.get('subclass_levels'):
         subclass_data['subclass_levels'] = level_changes
     # At the end all of indices are saved into the dictionary.
     dict_of_subclasses[entry['index']] = subclass_data
pprint.pprint(dict_of_subclasses)

In [ ]:
# This cell handles the establishment of the rule dictionary. 
url = "https://www.dnd5eapi.co/api/2014/rules"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_rules = {}
rule_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    rule_data = {
        'name' : name,
        'desc': "".join(preprocess_text(resp2['desc']))
    }
    # In some cases, the variables stored in the tables contain lists of dictionaries. So each dictionary is accessed and the name of the responding subsection is saved in a list.
    if resp2.get('subsections'):
         rule_data['subsection_in_rule_sections'] = [item['name'] for item in resp2['subsections']]

    dict_of_rules[entry['index']] = rule_data
pprint.pprint(dict_of_rules)

In [ ]:
# This cell handles the establishment of the spell dictionary.
url = "https://www.dnd5eapi.co/api/2014/spells"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_spells = {}
spell_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    spell_data = {
        'name' : name,
        'desc': "".join(preprocess_text(resp2['desc'])),
        'range': resp2['range'],
        'components':resp2['components'],
        'ritual':resp2['ritual'],
        'duration': resp2['duration'],
        'concentration': resp2['concentration'],
        'casting_time': resp2['casting_time'],
        'level': resp2['level'],
        'school_of_magic': [resp2['school']['name']],
        'classes': [item['name'] for item in resp2['classes']],
    }

    if resp2.get('material'):
         spell_data['material'] = resp2['material']

    if resp2.get('subclasses'):
        spell_data['subclasses'] = [item['name'] for item in resp2['subclasses']]

    if resp2.get('higher_level'):
        spell_data['higher_level'] = resp2['higher_level']

    if resp2.get('damage'):
        slot_level_damage = {}
        if 'damage_type' in resp2['damage']:
            spell_data['damage_type'] = [resp2['damage']['damage_type']['name']]
        if 'damage_at_slot_level' in resp2['damage']:
            for slot_level, damage in resp2['damage']['damage_at_slot_level'].items():
                slot_level_damage[slot_level] = damage
            spell_data['damage_at_slot_level'] = slot_level_damage 

    if resp2.get('atack_type'):
       spell_data['attack_type'] = resp2['attack_type']
    
    dict_of_spells[entry['index']] = spell_data
pprint.pprint(dict_of_spells)

In [ ]:
# This cell handles the different races a player can be:
url = "https://www.dnd5eapi.co/api/2014/races"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_races = {}
race_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)

    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    bonus_dict = {}
    for item in resp2['ability_bonuses']:
        bonus_dict[item['ability_score']['name']] = item['bonus']
    
    race_data = {
        'name' : name,
        'speed': resp2['speed'],
        'ability_bonuses': bonus_dict,
        'alignment': resp2['alignment'],
        'age': resp2['age'],
        'size': resp2['size'],
        'size_description': resp2['size_description'],
        'languages': [item['name'] for item in resp2['languages']],
        'language_description': resp2['language_desc'],
        'traits': [item['name'] for item in resp2['traits']],
    }

    if resp2.get('subraces') :
         race_data['subraces'] = [item['name'] for item in resp2['subraces']]

    if resp2.get('starting_proficiencies'):
        race_data['starting_proficiencies'] =  [item['name'] for item in resp2['starting_proficiencies']]


        
    dict_of_races[entry['index']] = race_data

pprint.pprint(dict_of_races)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/subraces"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_subraces = {}
subraces_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    subraces_data = {
        'name' : name,
        'desc': "".join(preprocess_text(resp2['desc'])),
        'race': resp2['race']['name'],
        'ability_bonuses': bonus_dict,
        'racial_traits': [item['name'] for item in resp2['racial_traits']],
    }

    if resp2.get('starting_proficiencies'):
        subraces_data['starting_proficiencies'] = [item['name'] for item in resp2['starting_proficiencies']]

    if resp2.get('languages'):
        subraces_data['languages'] = resp2['languages']

    if resp2.get('language_options'):
        languages = resp2['language_options']['from']['options']
        language_names = [lang['item']['name'] for lang in languages]
        subraces_data['language_options'] = language_names

    bonus_dict = {}
    for item in resp2['ability_bonuses']:
        bonus_dict[item['ability_score']['name']] = item['bonus'] 

    dict_of_subraces[entry['index']] = subraces_data
    
pprint.pprint(dict_of_subraces)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/proficiencies"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_proficiencies = {}
proficiency_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    proficiency_data = {
        'name' : name,
        'type' : resp2['type'],
    }

    if resp2.get('classes'):
         proficiency_data['classes'] = [item['name'] for item in resp2['classes']]

    if resp2.get('races'):
        proficiency_data['races'] = [item['name'] for item in resp2['races']]

    dict_of_proficiencies[entry['index']] = proficiency_data
pprint.pprint(dict_of_proficiencies)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/equipment"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()
dict_of_equipment = {}
equipment_data = {}

for entry in resp['results']:
    name_of_equip = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()
    
    equipment_data =  {
        'name' : name_of_equip,
        'equipment-category': resp2['equipment_category']['name'],
        'gear-category': resp2.get('gear_category',{}).get('name')
    }

    if resp2.get('desc'):
         equipment_data['desc'] = "".join(preprocess_text(resp2['desc']))

    if resp2.get('special'):
         equipment_data['special'] = resp2['special']

    if resp2.get('properties'):
         equipment_data['properties'] = [item['name'] for item in resp2['properties']]

    if resp2.get('contents'):
        equipment_data['contents'] = [{'name': item['item']['name']} for item in resp2['contents']]

    dict_of_equipment[entry['index']] = equipment_data
pprint.pprint(dict_of_equipment)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/features"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_features = {}
feature_data = {}

for entry in resp['results']:
    
    time.sleep(DELAY)

    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    feature_data = {
        'name' : entry['name'],
        'desc' : "".join(preprocess_text(resp2['desc'])),
        'class': resp2['class']['name'],
        'level': resp2['level']
    }

    if resp2.get('prerequisites'):
        feature_data['prerequisites'] =  resp2['prerequisites']
    
    dict_of_features[entry['index']] = feature_data
pprint.pprint(dict_of_features)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/magic-items"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_magic_items = {}
item_data = {}

for entry in resp['results']:
    name_of_item = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    item_data = {
        'name' : entry['name'],
        'desc' : "".join(preprocess_text(resp2['desc'])),
        'equipment-category': resp2['equipment_category']['name'],
        'rarity': resp2['rarity']['name'],
    }

    if resp2.get('variants'):
         item_data['variants'] = [item['name'] for item in resp2['variants']]

    dict_of_magic_items[entry['index']] = item_data
pprint.pprint(dict_of_magic_items)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/monsters"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_monsters = {}
monster_data = {}

for entry in resp['results']:
    name_of_monster = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    senses = {}
    for key, value in resp2['senses'].items():
        senses[key] = value
    
    movement = {}
    for key, value in resp2['speed'].items():
        movement[key] = value

    monster_data = {
        'name' : name_of_monster,
        'size': resp2['size'],
        'type':resp2['type'],
        'alignment': resp2['alignment'],
        'hit_points':resp2['hit_points'],
        'hit_dice':resp2['hit_dice'],
        'hit_points_roll': resp2['hit_points_roll'],
        'speed': movement,
        'strength': resp2['strength'],
        'dexterity':resp2['dexterity'],
        'constitution': resp2['constitution'],
        'intelligence': resp2['intelligence'],
        'wisdom': resp2['wisdom'],
        'charisma': resp2['charisma'],
        'senses': senses,
        'languages': resp2['languages'],
        'challenge_rating': resp2['challenge_rating'],
        'proficiency_bonus': resp2['proficiency_bonus'],
        'gained_experience': resp2['xp']
    }

    if resp2.get('armor_class'):
       armor_class = {}
       for item in resp2['armor_class']:
            armor_class[item['type']] = item['value']
       monster_data['armor_class'] = armor_class

    if resp2.get('damage_vulnerabilities'):
        monster_data['damage_vulnerabilites'] = resp2['damage_vulnerabilities']

    if resp2.get('damage_resistances'):
        monster_data['damage_resistances'] = resp2['damage_resistances']
    
    if resp2.get('damage_immunities'):
        monster_data['damage_immunities'] = resp2['damage_immunities']

    if resp2.get('condition_immunities'):
        monster_data['condition_immunities'] = [item['name'] for item in resp2['condition_immunities']]

    if resp2.get('special_abilites'):
        special = {}
        for items in resp2['special_abilities']:
            special['name'] = items['name']
            special['desc'] = items['desc']
            if 'damage' in items:
                special['damage'] = items['damage']
            if 'dc' in items:
                dc = {}
                dc['name'] = items['dc']['dc_type']['name']
                dc['value'] = items['dc']['dc_value']
                special['dc'] = dc
        monster_data['special_abilities'] = special
    
    if resp2.get('actions'):
        for items in resp2['actions']:
            actions = {}
            actions['name'] = items['name']
            actions['desc'] = items['desc']
        monster_data['actions'] = actions

    if resp2.get('legendary_actions'):
        legendary = {}
        for items in resp2['legendary_actions']:
            legendary['name'] = items['name']
            legendary['action_desc'] = items['desc']
        monster_data['legendary_actions'] = legendary

    if resp2.get('forms'):
        monster_data['forms'] = [item['name'] for item in resp2['forms']]

    if resp2.get('reactions'):
        reactions = {}
        for item in resp2['reactions']:
             reactions['name'] = item['name']
             reactions['desc'] = item['desc']
        monster_data['reactions'] = reactions
    
    proficiency_monster = {}
    if resp2.get('proficiencies'):
        for items in resp2['proficiencies']:
            proficiency_monster[items['proficiency']['name']] = items['value']
        monster_data['proficiencies'] = proficiency_monster

    dict_of_monsters[entry['index']] = monster_data
# pprint.pprint(dict_of_monsters)



In [ ]:
url = "https://www.dnd5eapi.co/api/2014/equipment-categories"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_equipment_categories = {}
category_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    category_data = {
        'name' : name
    }

    if resp2.get('equipment'):
        category_data['type'] = [item['name'] for item in resp2['equipment']]

   
    dict_of_equipment_categories[entry['index']] = category_data

pprint.pprint(dict_of_equipment_categories)

In [ ]:
url = "https://www.dnd5eapi.co/api/2014/backgrounds"
response = requests.request("GET", url, headers=headers, data=payload)
resp = response.json()

dict_of_backgrounds = {}
background_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(DELAY)
    
    response2 = requests.request("GET", url+f"/{entry['index']}", headers=headers, data=payload)
    resp2 = response2.json()

    background_data = {
        'name' : name
    }

    if resp2.get('starting_proficiencies'):
        background_data['starting_proficiencies'] = [item['name'] for item in resp2['starting_proficiencies']]

    if resp2.get('language_options'):
        background_data['language_options'] = resp2['language_options']['choose']

    if resp2.get('starting_equipment'):
        background_data['starting_equipment'] = [item['equipment']['name'] for item in resp2['starting_equipment']]

    if resp2.get('starting_equipment_options'):
        starting_equip_options = {}
        starting_equip_options['choose'] = [item['choose'] for item in resp2['starting_equipment_options']]
        starting_equip_options['equipment_options'] = [item['from']['equipment_category']['name'] for item in resp2['starting_equipment_options']]
        background_data['starting_equipment_options'] = starting_equip_options

    personality = { 'options':[] } 
    personality['amount_of_options'] = resp2['personality_traits']['choose'] 
    for item in resp2['personality_traits']['from']['options']: 
        personality['options'].append(item['string'])
    background_data['personality_traits'] = personality

    if resp2.get('feature'):
        feat_dict = {}
        feat_dict['name'] = resp2['feature']['name']
        feat_dict['desc'] =  "".join(resp2['feature']['desc'])
        background_data['feature'] = feat_dict

    if resp2.get('ideals'):
        ideals = {}
        ideals['choose'] = resp2['ideals']['choose']
        ideal_option = {}
        ideals['possible_ideals'] = []
        for item in resp2['ideals']['from']['options']:
            ideal_option['desc'] = item['desc']
            ideal_option['alignments'] = [item['name'] for item in item['alignments']]
            ideals['possible_ideals'].append(ideal_option)
        background_data['ideals'] = ideals
    
    if resp2.get('bonds'):
        bonds = {}
        bonds['choose'] = resp2['bonds']['choose']
        bonds['bond_options'] = [item['string'] for item in resp2['bonds']['from']['options']]
        background_data['bonds'] = bonds

    if resp2.get('flaws'):
        flaws = {}
        flaws['choose'] = resp2['flaws']['choose']
        flaws['flaw_options'] = [item['string'] for item in resp2['flaws']['from']['options']]
        background_data['flaws'] = flaws

    dict_of_backgrounds[entry['index']] = background_data

pprint.pprint(dict_of_backgrounds)


In [ ]:
# Now all dicts will be saved into the api_data.json. In order to structure the data in the json itself, a new dict is constructed, saving each dictionary under a thematically responding key. 
file_path = 'api_data/api_data.json'
json_dict = {
            'rules': dict_of_rules,
            'rule_sections': dict_of_rule_sections,
            'races': dict_of_races,
            'subraces': dict_of_subraces,
            'classes': dict_of_classes,
            'subclasses': dict_of_subclasses,
            'skills': dict_of_skills,
            'feats': dict_of_feats,
            'languages': dict_of_languages,
            'ability_scores': dict_of_ability_scores,
            'traits': dict_of_traits,
            'proficiencies': dict_of_proficiencies,
            'features': dict_of_features,
            'example_character_background': dict_of_backgrounds,
            'conditions': dict_of_conditions,
            'equipment': dict_of_equipment,
            'equipment_categories': dict_of_equipment_categories,
            'weapon_properties': dict_of_weapon_properties,
            'magic_items': dict_of_magic_items,
            'magic_schools': dict_of_magic_schools,
            'damage_types': dict_of_damage_types,
            'spells': dict_of_spells,
            'monsters': dict_of_monsters
        }

with open(file_path, 'w') as f:
    # The previously constructed dictionary is written to the json file:
    json.dump(json_dict,
        indent=4, # For better readability and visible structure four indents are added.
        ensure_ascii=False, # This is set to false, so f.ex. apostrophes aren't converted.
        fp=f
    )
    f.close()